## scNym: https://github.com/calico/scnym 

In [1]:
import scnym
from scnym.api import scnym_api
import torch
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import scanpy as sc

In [2]:
def split_data(data_path: str, 
               fold: int=1,
               folds: int=5,
               batch_key: str="patientID", 
               label_key: str="cell_type",
               seed: int=42,
               HVG: bool=True,
               HVGs: int=2000):
    
    adata = sc.read(data_path, cache=True)

    adata.obs["batch"] = adata.obs[batch_key]

    # Ensure reproducibility
    def rep_seed(seed):
        # Check if a GPU is available
        if torch.cuda.is_available():
            # Set the random seed for PyTorch CUDA (GPU) operations
            torch.cuda.manual_seed(seed)
            # Set the random seed for all CUDA devices (if multiple GPUs are available)
            torch.cuda.manual_seed_all(seed)
        
        # Set the random seed for CPU-based PyTorch operations
        torch.manual_seed(seed)
        
        # Set the random seed for NumPy
        np.random.seed(seed)
        
        # Set the random seed for Python's built-in 'random' module
        random.seed(seed)
        
        # Set the random seed for TensorFlow
        tf.random.set_seed(seed)
        
        # Set CuDNN to deterministic mode for PyTorch (GPU)
        torch.backends.cudnn.deterministic = True
        
        # Disable CuDNN's benchmarking mode for deterministic behavior
        torch.backends.cudnn.benchmark = False

    rep_seed(seed)

    # Initialize Stratified K-Fold
    stratified_kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    # Iterate through the folds
    adata_original = adata
    test_adata_original = adata
    fold_counter = 0
    for train_index, test_index in stratified_kfold.split(adata_original.X, adata_original.obs[label_key]):
        fold_counter += 1
        
        if fold_counter == fold:

            adata = adata_original[train_index, :]
            test_adata = test_adata_original[test_index, :]

            if HVG:
                sc.pp.highly_variable_genes(adata, n_top_genes=HVGs, flavor="cell_ranger")
                test_adata = test_adata[:, adata.var["highly_variable"]]
                adata = adata[:, adata.var["highly_variable"]]
            
            return adata, test_adata


In [ ]:
data_path = "../../../data/processed/data_for_evaluating_cell_type_annotation/Zheng68k.h5ad"

# Iterate through folds
results = None
for fold_idx in range(5):
    fold = fold_idx + 1

    adata_train, adata_test = split_data(data_path=data_path, fold=fold)

    # Train model
    scnym_api(
        adata=adata_train,
        task='train',
        groupby='cell_type',
        out_path='./scnym_outputs',
        config='no_new_identity',
    )

    # Predict test data
    scnym_api(
        adata=adata_test,
        task='predict',
        key_added='scNym',
        config='no_new_identity',
        trained_model='./scnym_outputs'
    )

    # Save results in dataframe
    if fold == 1:
        results = pd.DataFrame({"pred": adata_test.obs['scNym'].to_list(), 
                                "true_label": adata_test.obs["cell_type"].to_list(), 
                                "fold": [fold]*adata_test.obs['scNym'].shape[0]})
    else:   
        dataframe_temp = pd.DataFrame({"pred": adata_test.obs['scNym'].to_list(), 
                                    "true_label": adata_test.obs["cell_type"].to_list(), 
                                    "fold": [fold]*adata_test.obs['scNym'].shape[0]})
        results = pd.concat([results,dataframe_temp], axis=0)

# Reset index
results.reset_index(drop=True, inplace=True)

In [3]:
results = pd.read_csv('results/scNym_hvgs_output.csv', index_col=0)

In [4]:
data_path = "../../../data/processed/data_for_evaluating_cell_type_annotation/Zheng68k.h5ad"

# Iterate through folds
fold = 5

adata_train, adata_test = split_data(data_path=data_path, fold=fold)

# Train model
scnym_api(
    adata=adata_train,
    task='train',
    groupby='cell_type',
    out_path='./scnym_outputs',
    config='no_new_identity',
)

# Predict test data
scnym_api(
    adata=adata_test,
    task='predict',
    key_added='scNym',
    config='no_new_identity',
    trained_model='./scnym_outputs'
)

# Save results in dataframe
if fold == 1:
    results = pd.DataFrame({"pred": adata_test.obs['scNym'].to_list(), 
                            "true_label": adata_test.obs["cell_type"].to_list(), 
                            "fold": [fold]*adata_test.obs['scNym'].shape[0]})
else:   
    dataframe_temp = pd.DataFrame({"pred": adata_test.obs['scNym'].to_list(), 
                                "true_label": adata_test.obs["cell_type"].to_list(), 
                                "fold": [fold]*adata_test.obs['scNym'].shape[0]})
    results = pd.concat([results,dataframe_temp], axis=0)

results.reset_index(drop=True, inplace=True)

c:\Users\user\Documents\GitHub\Masters_Thesis\AnnotationEnv\lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
c:\Users\user\Documents\GitHub\Masters_Thesis\AnnotationEnv\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:252: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
c:\Users\user\Documents\GitHub\Masters_Thesis\AnnotationEnv\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:544: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['hvg'] = {'flavor': flavor}


No CUDA device found.
Computations will be performed on the CPU.
Add a CUDA compute device to improve speed dramatically.

No unlabeled data was found.
Did you forget to set some examples as `"Unlabeled"`?
Proceeding with purely supervised training.

X:  (54864, 2000)
y:  (54864,)
Not weighting classes and not balancing classes.
Training...
Epoch 0/99
----------
Iter :  0
running_loss :  0.010007387027144432
running_acc  :  0.0703125
corrects: 18.000000 | total: 256.000000
Iter :  100
running_loss :  0.0037538731207905135
running_acc  :  0.6531946163366337
corrects: 16889.000000 | total: 25856.000000
train Loss : 0.0034
train Acc : 0.6786
TRAIN EPOCH corrects: 30157.000000 | total: 44439.000000
val Loss : 0.0028
val Acc : 0.7485
VAL EPOCH corrects: 3696.000000 | total: 4938.000000
Epoch 1/99
----------
Iter :  0
running_loss :  0.0022753924131393433
running_acc  :  0.8125
corrects: 208.000000 | total: 256.000000
Iter :  100
running_loss :  0.002692130182196598
running_acc  :  0.7427289

c:\Users\user\Documents\GitHub\Masters_Thesis\AnnotationEnv\lib\site-packages\scnym\api.py:427: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['scNym_train_results'] = results


No CUDA device found.
Computations will be performed on the CPU.
Add a CUDA compute device to improve speed dramatically.

Loaded model predicting 11 classes from 2000 features
['CD14+ Monocyte' 'CD19+ B' 'CD34+' 'CD4+ T Helper2' 'CD4+/CD25 T Reg'
 'CD4+/CD45RA+/CD25- Naive T' 'CD4+/CD45RO+ Memory' 'CD56+ NK'
 'CD8+ Cytotoxic T' 'CD8+/CD45RA+ Naive Cytotoxic' 'Dendritic']
Building a classification matrix...
Gene names match exactly, returning input.
Predicting cell types...


Finding cell types: 100%|██████████| 14/14 [00:00<00:00, 31.77it/s]


Extracting model embeddings...


c:\Users\user\Documents\GitHub\Masters_Thesis\AnnotationEnv\lib\site-packages\scnym\api.py:550: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[config['key_added']] = names


In [6]:
# Save results
results.to_csv('results/scNym_hvgs_output.csv', index=True)

In [7]:
# Extract the unique labels
unique_labels1 = np.unique(results.true_label)
unique_labels2 = np.unique(results.pred)
unique_labels = np.unique(np.concatenate([unique_labels1,unique_labels2]))

# Convert string labels to numerical labels
label_encoder_temp = LabelEncoder()
label_encoder_temp.fit(unique_labels)
y_true = label_encoder_temp.transform(results.true_label)
y_pred = label_encoder_temp.transform(results.pred)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7645634961139708
